# 1. Requierments and Packages

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.0/872.0 kB 15.6 MB/s eta 0:00:00


In [ ]:
!pip install scikit-learn lap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628948 sha256=75e260836b451dbe24bc6dcf2b6ae786d0086507ec441e913e444bf618f83e08
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built lap


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
import os
from google.colab import files

# 2. Preprocesses and setting Parameters

## 2.1 Hyperparameters

In [ ]:
class Config:
  BotSort_Yaml = """
tracker_type: bytetrack # tracker type, ['botsort', 'bytetrack']
track_high_thresh: 0.6 # threshold for the first association
track_low_thresh: 0.3 # threshold for the second association
new_track_thresh: 0.85 # threshold for init new track if the detection does not match any tracks
track_buffer: 100 # buffer to calculate the time when to remove tracks
match_thresh: 0.6 # threshold for matching tracks
fuse_score: True # Whether to fuse confidence scores with the iou distances before matching
min_box_area: 10  # threshold for min box areas(for tracker evaluation, not used for now)
# BoT-SORT settings
gmc_method: sparseOptFlow # method of global motion compensation
# ReID model related thresh (not supported yet)
proximity_thresh: 0.6
appearance_thresh: 0.6
with_reid: True
"""
  Bytetrack_Yaml = """
  tracker_type: bytetrack # tracker type, ['botsort', 'bytetrack']
  track_high_thresh: 0.35 # threshold for the first association
  track_low_thresh: 0.025 # threshold for the second association
  new_track_thresh: 0.8 # threshold for init new track if the detection does not match any tracks
  track_buffer: 300 # buffer to calculate the time when to remove tracks
  match_thresh: 0.8 # threshold for matching tracks
  fuse_score: True # Whether to fuse confidence scores with the iou distances before matching
  min_box_area: 1  # threshold for min box areas(for tracker evaluation, not used for now)
  """

  output_path = '/content/output_video.mp4'

  tracker="botsort.yaml"

  imgsz=1280

  conf=0.2

  iou=0.7

  font_scale = 0.9

  thickness = 2

## 2.2 Setting BotSort yaml

In [ ]:
# default values of botsort
#tracker_type: bytetrack # tracker type, ['botsort', 'bytetrack']
#track_high_thresh: 0.5 # threshold for the first association
#track_low_thresh: 0.1 # threshold for the second association
#new_track_thresh: 0.6 # threshold for init new track if the detection does not match any tracks
#track_buffer: 30 # buffer to calculate the time when to remove tracks
#match_thresh: 0.8 # threshold for matching tracks
#fuse_score: True # Whether to fuse confidence scores with the iou distances before matching
#min_box_area: 10  # threshold for min box areas(for tracker evaluation, not used for now)
# BoT-SORT settings
#gmc_method: sparseOptFlow # method of global motion compensation
# ReID model related thresh (not supported yet)
#proximity_thresh: 0.5
#appearance_thresh: 0.25
#with_reid: False

In [ ]:
def set_BotSort_yaml():
  # Write to YAML file
  with open("/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/trackers/botsort.yaml", "w") as file:
      file.write(Config.BotSort_Yaml)


In [ ]:
def view_BotSort_yaml():
  # View the contents of the botsort.yaml file
  !cat /usr/local/lib/python3.10/dist-packages/ultralytics/cfg/trackers/botsort.yaml

## 2.3 Setting ByteTrack yaml

In [ ]:
# default values of bytetrack
#tracker_type: bytetrack # tracker type, ['botsort', 'bytetrack']
#track_high_thresh: 0.5 # threshold for the first association
#track_low_thresh: 0.1 # threshold for the second association
#new_track_thresh: 0.6 # threshold for init new track if the detection does not match any tracks
#track_buffer: 30 # buffer to calculate the time when to remove tracks
#match_thresh: 0.8 # threshold for matching tracks
#fuse_score: True # Whether to fuse confidence scores with the iou distances before matching
#min_box_area: 10  # threshold for min box areas(for tracker evaluation, not used for now)

In [ ]:
def set_ByteTrack_yaml():
  # Write to YAML file
  with open("/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/trackers/bytetrack.yaml", "w") as file:
      file.write(Config.Bytetrack_Yaml)


In [ ]:
def view_ByteTrack_yaml():
  # View the contents of the bytetrack.yaml file
  !cat /usr/local/lib/python3.10/dist-packages/ultralytics/cfg/trackers/bytetrack.yaml

# 3. Image Enhancements Functions

In [ ]:
def blur(frame):
  blurred_frame = cv2.blur(frame, (5, 5))
  return blurred_frame

def gaussian_blur(frame):
  blurred_frame = cv2.GaussianBlur(image, (5, 5), 0)
  return blurred_frame

# 4. Object detection and Tracking

In [ ]:
def load_test_videos():
  # Input video path
  video_path_vally = "./vally_test2.mp4"
  video_path_traffic = "./test.MOV"
  video_path_dogs = "./person&dogs.mp4"

  cap = cv2.VideoCapture(video_path_dogs)

  # Check if video capture initialized successfully
  if not cap.isOpened():
      print("Error: Could not open video.")
      exit()

  return cap

def make_output_video(cap):

  # Get video properties
  fps = cap.get(cv2.CAP_PROP_FPS)
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frame_count = 0

  # Output video path
  output_video_path = Config.output_path

  # Define the codec and create VideoWriter object to save processed frames as a video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

  return output_video_path, out

def load_model():
  # Load the YOLO model
  model = YOLO("yolov8x.pt")
  return model

def draw_customized_boxes(result, frame):
   # Get the boxes, class names, and identities from the results
        for r in result[0].boxes:
            box = r.xyxy[0]  # Extract bounding box coordinates
            label=""
            #conf = r.conf[0]  # Extract confidence score
            #cls = int(r.cls[0])  # Extract class ID

            if r.id is not None:  # Check if the ID is available
                obj_id = int(r.id[0])  # Get object ID
                label = f"id:{obj_id}"  # Combine label and ID
                #label = f"{model.names[cls]} {conf:.2f} id:{obj_id}"  # Combine label and ID

            # Draw the bounding box without confidence score
            color = (255, 0, 0)  # Blue color for bounding boxes
            cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), color, 2)

            # Get text size to create a background rectangle
            font_scale = 0.9
            thickness = 2
            text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)[0]
            text_x, text_y = int(box[0]), int(box[1]) - 10

            # Define the coordinates for the background rectangle
            background_tl = (text_x, text_y - text_size[1] - 5)  # Top-left corner
            background_br = (text_x + text_size[0] + 5, text_y + 5)  # Bottom-right corner

            # Draw the blue background rectangle
            cv2.rectangle(frame, background_tl, background_br, (255, 0, 0), -1)  # Blue background

            # Draw the label text on top of the background rectangle
            cv2.putText(frame, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)


def detect_and_track_objects(model, cap, out):
  try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess frames if needed
        blurred_frame = blur(frame)

        # Apply YOLO tracking
        result = model.track(blurred_frame, persist=True, tracker="botsort.yaml", imgsz=1280, conf=0.2, iou=0.7)

        draw_customized_boxes(result, frame)

        #cv2_imshow(frame)

        # Write the processed frame to the output video file
        out.write(frame)

        if cv2.waitKey(25) & 0xFF == ord("q"):
            break

  except KeyboardInterrupt:
      print("Processing interrupted by user.")
  finally:
      # Release resources properly even if the processing is interrupted
      cap.release()
      out.release()
      cv2.destroyAllWindows()


def download_output_video(output_video_path):
    # Ensure the video is downloadable if possible
    if os.path.exists(output_video_path):
        files.download(output_video_path)


In [ ]:
def task_pipeline():
  set_BotSort_yaml()
  view_BotSort_yaml()
  cap = load_test_videos()
  path, out = make_output_video(cap)
  model = load_model()
  detect_and_track_objects(model, cap, out)
  download_output_video(path)

In [ ]:
task_pipeline()


# default values of botsort
tracker_type: bytetrack # tracker type, ['botsort', 'bytetrack']
track_high_thresh: 0.6 # threshold for the first association
track_low_thresh: 0.3 # threshold for the second association
new_track_thresh: 0.85 # threshold for init new track if the detection does not match any tracks
track_buffer: 100 # buffer to calculate the time when to remove tracks
match_thresh: 0.6 # threshold for matching tracks
fuse_score: True # Whether to fuse confidence scores with the iou distances before matching
min_box_area: 10  # threshold for min box areas(for tracker evaluation, not used for now)
# BoT-SORT settings
gmc_method: sparseOptFlow # method of global motion compensation
# ReID model related thresh (not supported yet)
proximity_thresh: 0.6
appearance_thresh: 0.6
with_reid: True

0: 736x1280 3 persons, 2 dogs, 9471.7ms
Speed: 6.8ms preprocess, 9471.7ms inference, 3.0ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 3 persons, 2 dogs, 10253.6ms
Speed: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Change and adjust Kalman filter parameters

In [ ]:
# Use the find command to search for kalman_filter.py file in the entire file system
!find / -name "kalman_filter.py" 2>/dev/null

/usr/local/lib/python3.10/dist-packages/ultralytics/trackers/utils/kalman_filter.py
/usr/local/lib/python3.10/dist-packages/filterpy/kalman/kalman_filter.py
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/kalman_filter.py
